In [4]:
import subprocess
def ExecuteCmd(command_str, output_file='output_cmd/output.txt'):
  
    with open(output_file, 'w') as f:
        result = subprocess.run(
            command_str,
            shell=True,
            stderr=f, 
            stdout=f,
            text=True,
            executable='/bin/bash'
        )
    return f"Output saved to {output_file}"

In [ ]:
from dotenv import load_dotenv
import os
from google import genai
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client()

In [ ]:
import json

# Path to your JSON file
file_path = "Datasets/network_analysis_2/na2.json"

# Load the JSON content
with open(file_path, "r") as f:
    data = json.load(f)

# Extract parts from the JSON
scenario = data["scenario"]
tools = data["tools"]
questions = data["questions"]
file=data["files"]



In [8]:
# the first thought or command for each quest based on prompt
def Tool1(history):
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents="\n".join([str(h) for h in history])
    )
    #history.append({"role": "model", "text": [f"Command generated by the model:\n{response.text}"]})
    
    return response.text

#takes command from tool1 or tool2_action and returns thought
def tool2_thought(history, thought, command):
    with open('output_cmd/output.txt', 'r') as f:
        output = f.read()

    if len(output) > 500:
        output = output[:500] + "\n...[truncated]..." #for large output file we only take the first 500 characters 

    history.append({"previous Thought": [thought],"previous command": [command], "previous command output": [output]})

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents="\n".join([str(h) for h in history])
    )
    return response.text

def tool2_action(history):
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents="\n".join([str(h) for h in history])
    )
    return response.text


In [9]:
from Prompts import prompter
i = -1
nb_questions = len(questions)
history = []
final_answers= ["Answer not found"] * nb_questions  # Initialize with default answer
previous_input = None
output0 = None  # Initialize 
output1 = None
# Main control loop
while i < nb_questions:
    user_input = input("\nChoose:\n1. Next Question\n2. Analyze/Continue\n3. Exit\n>>> ")
    if previous_input is None and user_input not in ["1", "3"]:
        print("❌ You must start with option 1.")
        continue

    
    if user_input == previous_input and user_input == "1":
        print("❌ You already chose 1. You must alternate between 1 and 2.")
        continue

    previous_input = user_input  # Store the last input for validation

    if user_input == "1":
        if i == nb_questions - 1:
            print("✅ Task finished.")
            break
        i += 1
        prompt = prompter.prompt_tool1_thought(i, scenario, tools, file, questions)    #first prompt for tool1 thought
        history = [{"role": "system", "text": [prompt]}]  # start fresh for new question 
        output0 = Tool1(history)#first thought
        print(f"🧠 Thought generated for question {i+1} by tool 1: {output0}")
        prompt = prompter.prompt_tool1_action(i, scenario, tools, file, questions,output0)    #first prompt for tool1 
        history = [{"role": "system", "text": [prompt]}]  # change for
        output1 = Tool1(history)# first command
        print(f"🛠️ Command generated for question {i+1} by tool 1: {output1}, press 2 to execute")
        ExecuteCmd(output1)  # Execute the command generated by Tool1
        #we should modify the history here for the tool2: new history becomes empty -> receives the new prompt -> receives command
        

    elif user_input == "2":
        prompt = prompter.prompt_tool2_thought(i, scenario, tools, file, questions)
        history[0]["text"]=[prompt]
        output0 = tool2_thought(history, output0, output1) #genrate tool2 thought
        print(f"🧠 Thought generated for question {i+1} by tool 2: {output0}")
        prompt = prompter.prompt_tool2_action(i, scenario, tools, file, questions, output0)
        history[0]["text"]=[prompt]
        output1 =tool2_action(history)
    
        print(f"📜 {history}")
        if "final answer:" in output1.lower():
            final_answers[i] = output1.lower().split("final answer:")[-1].strip()
            print(f"Answer to the question {i+1}: {final_answers[i]}, click 1 to move to next question")
            
        else:
            print(f"🛠️ New command: {output1}, press 2 to execute it ")
            ExecuteCmd(output1)
        
        #if final result append to list else exucute cmd
    elif user_input == "3":
        print("⛔ Stopped by user.")
        break

    else:
        print("⚠️ Invalid input. Please enter 1, 2, or 3.")


❌ You must start with option 1.
🧠 Thought generated for question 1 by tool 1: The goal is to identify the source IP conducting the port scan. I need to analyze the provided PCAP file to detect patterns indicative of a port scan. This typically involves identifying a single source IP that is attempting to connect to a large number of different ports on one or more destination systems. I will look for an IP sending out multiple connection attempts (e.g., SYN packets) to various ports to determine the scanning IP.
🛠️ Command generated for question 1 by tool 1: `tshark -r Datasets/network_analysis_2/BTLOPortScan.pcap -Y "tcp.flags.syn==1" -T fields -e ip.src | sort | uniq -c | sort -nr`, press 2 to execute
🧠 Thought generated for question 1 by tool 2: The previous command failed with an unexpected "command not found" error, which seems to be an artifact of the execution environment rather than an error in the `tshark` command syntax itself. The original command's logic, which counts source

In [10]:
print(final_answers)

['10.251.96.4', '1-152', 'tcp connect', 'Answer not found', 'Answer not found', 'Answer not found', 'Answer not found', 'Answer not found', 'ssh', 'Answer not found']


In [11]:
#can be transformed to a function to handle both prompt types 
base_name = os.path.splitext(os.path.basename(file_path))[0]

output_file = os.path.join("Evaluation","tool_augmented","ReAct", f"{base_name}.txt")#change path based on type of prompting

with open(output_file, 'w') as f:
    for answer in final_answers:
        f.write(answer + "\n")

print(f"Answers saved to {output_file}")

Answers saved to Evaluation/tool_augmented/ReAct/na2.txt
